# Cross-validation for iris

## Stacked Generalization (Stacking)

Stacked generalization (or stacking) (Wolpert, 1992) is a different way of combining multiple models, that introduces the concept of a meta learner. Although an attractive idea, it is less widely used than bagging and boosting. Unlike bagging and boosting, stacking may be (and normally is) used to combine models of different types. The procedure is as follows:

1) Split the training set into two disjoint sets.

2) Train several base learners on the first part.

3) Test the base learners on the second part.

4) Using the predictions from 3) as the inputs, and the correct responses as the outputs, train a higher level learner.

Note that steps 1) to 3) are the same as cross-validation, but instead of using a winner-takes-all approach, we combine the base learners, possibly nonlinearly.

In [120]:
from sklearn import datasets
from sklearn.utils.validation import check_random_state
from stacked_generalization.lib.stacking import StackedClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.linear_model import Ridge
from sklearn.cross_validation import StratifiedKFold
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np

In [121]:
iris = datasets.load_iris()
rng = check_random_state(0)
perm = rng.permutation(iris.target.size)
iris.data = iris.data[perm]
iris.target = iris.target[perm]

In [133]:
iris.data

array([[5.8, 2.8, 5.1, 2.4],
       [6. , 2.2, 4. , 1. ],
       [5.5, 4.2, 1.4, 0.2],
       [7.3, 2.9, 6.3, 1.8],
       [5. , 3.4, 1.5, 0.2],
       [6.3, 3.3, 6. , 2.5],
       [5. , 3.5, 1.3, 0.3],
       [6.7, 3.1, 4.7, 1.5],
       [6.8, 2.8, 4.8, 1.4],
       [6.1, 2.8, 4. , 1.3],
       [6.1, 2.6, 5.6, 1.4],
       [6.4, 3.2, 4.5, 1.5],
       [6.1, 2.8, 4.7, 1.2],
       [6.5, 2.8, 4.6, 1.5],
       [6.1, 2.9, 4.7, 1.4],
       [4.9, 3.1, 1.5, 0.1],
       [6. , 2.9, 4.5, 1.5],
       [5.5, 2.6, 4.4, 1.2],
       [4.8, 3. , 1.4, 0.3],
       [5.4, 3.9, 1.3, 0.4],
       [5.6, 2.8, 4.9, 2. ],
       [5.6, 3. , 4.5, 1.5],
       [4.8, 3.4, 1.9, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [6.2, 2.8, 4.8, 1.8],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.8, 1.9, 0.4],
       [6.2, 2.9, 4.3, 1.3],
       [5. , 2.3, 3.3, 1. ],
       [5. , 3.4, 1.6, 0.4],
       [6.4, 3.1, 5.5, 1.8],
       [5.4, 3. , 4.5, 1.5],
       [5.2, 3.5, 1.5, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [6.4, 2

In [123]:
bclf = LogisticRegression(random_state=1)

In [124]:
clfs = [RandomForestClassifier(n_estimators=40, criterion = 'gini', random_state=1),
        ExtraTreesClassifier(n_estimators=30, criterion = 'gini', random_state=3),
        GradientBoostingClassifier(n_estimators=25, random_state=1),
        GradientBoostingClassifier(n_estimators=30, random_state=2),
        #GradientBoostingClassifier(n_estimators=30, random_state=3),
        KNeighborsClassifier(),
        RidgeClassifier(random_state=1),
        Ridge(),
        TSNE(n_components=2)
        ]


K-fold validation seeks to avoid having the random choice of training data vs. validation data affect our expectations of predictive performance. In other words, if we got “lucky” and chose a train/validation split of the data that gave excellent predictions on the validation, and we reported that, it is likely that future predictions won’t be as accurate. To address this, multiple repeats are run and averaged, where the repeats are the “k-folds” of the data. A fold is just one split of the original data into train/validation/test.

In [125]:
sc = StackedClassifier(bclf,
                       clfs,
                       n_folds=5,
                       verbose=0,
                       stack_by_proba=True,
                       oob_score_flag=True,
                       )

Stacking is an ensemble learning technique to combine multiple classification models via a meta-classifier. The individual classification models are trained based on the complete training set; then, the meta-classifier is fitted based on the outputs -- meta-features -- of the individual classification models in the ensemble. The meta-classifier can either be trained on the predicted class labels or probabilities from the ensemble.

In [126]:
gb = GradientBoostingClassifier(n_estimators=25, random_state=1)

The model giving the best validation results is selected as the best choice. Then, the model is trained on a number (denoted as k) of different subsets of the data, and the expected performance in the future is given by mean and standard deviation of the k validation results on the validation data, and a final check is done with the test data. Test data results are not used to tune the model!


In [127]:
sc_score = 0
gb_score = 0
n_folds = 5

In [128]:
for train_idx, test_idx in StratifiedKFold(iris.target, n_folds):
    xs_train = iris.data[train_idx]
    y_train = iris.target[train_idx]
    xs_test = iris.data[test_idx]
    y_test = iris.target[test_idx]

    sc.fit(xs_train, y_train)
    print('oob_score: {0}'.format(sc.oob_score_))
    sc_score += sc.score(xs_test, y_test)
    gb.fit(xs_train, y_train)
    gb_score += gb.score(xs_test, y_test)    

oob_score: 0.9333333333333333
oob_score: 0.975
oob_score: 0.9416666666666667
oob_score: 0.9416666666666667
oob_score: 0.9666666666666667


In [129]:
sc_score /= n_folds
print('Stacked Classfier score: {0}'.format(sc_score))
gb_score /= n_folds
print('Gradient Boosting Classfier score: {0}'.format(gb_score))

Stacked Classfier score: 0.9533333333333334
Gradient Boosting Classfier score: 0.96
